In [1]:
# import necessary packages
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from plotnine import *
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler # Z-score variables
from sklearn.preprocessing import MinMaxScaler # Min-Max Normalization

from sklearn.model_selection import train_test_split # simple TT split cv

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import os
import numpy as np
from scipy.interpolate import BSpline, make_interp_spline

Overview: Plots two strokes at the origin of the same figure

Details:
    - Currently only works with unimanual right controller strokes.
        - Test data: Sub 16 Sess 2 & 3 for pan down
    - Only traces the first trial.
    - Only plots the right controller's data points.

Requirements: 
    - Must edit variables 'path1' and 'path2' to your own path of two data files


In [2]:
#Removes rows where none of the triggers are being pulled and all trials that are not the specified trial
def drop_df(df, trial_num):
    df.drop(df[(df['trigger_pull_amount_left'] == 0) & (df['trigger_pull_amount_right'] == 0)].index, inplace=True)
    df.drop(df[(df['gesture_counter_UI']) != trial_num].index, inplace=True)
    df.reset_index(drop=True, inplace=True)

#Edit here
path1 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub16\\Freeform_Sub16_Sess2\\cleaned_session_F_PanDown_subjID_16_06-15-23_10-53-50.csv'
path2 = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData\\Sub16\\Freeform_Sub16_Sess3\\cleaned_session_F_PanDown_subjID_16_06-20-23_10-46-47.csv'

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

drop_df(df1, 1)
drop_df(df2, 1)

fig = go.Figure()

#List of X, Y, Z points that make up the original strokes.
x_original_1 = np.array(df1['r_controller_translation_x'])
y_original_1 = np.array(df1['r_controller_translation_y'])
z_original_1 = np.array(df1['r_controller_translation_z'])
x_original_2 = np.array(df2['r_controller_translation_x'])
y_original_2 = np.array(df2['r_controller_translation_y'])
z_original_2 = np.array(df2['r_controller_translation_z'])

'''Smoothing the curve'''
# The number of control points and knots
k = 3  # degree of the B-spline
t1 = np.linspace(0, 1, len(x_original_1))
t2 = np.linspace(0, 1, len(x_original_2))

# Create the B-spline representation for each dimension
spl_x1 = make_interp_spline(t1, x_original_1, k=k)
spl_y1 = make_interp_spline(t1, y_original_1, k=k)
spl_z1 = make_interp_spline(t1, z_original_1, k=k)
spl_x2 = make_interp_spline(t2, x_original_2, k=k)
spl_y2 = make_interp_spline(t2, y_original_2, k=k)
spl_z2 = make_interp_spline(t2, z_original_2, k=k)

# Evaluate the B-spline over a dense set of points for a smooth trajectory
dense_t1 = np.linspace(0, 1, 5 * len(x_original_1))  # points in the smoothed curve; can be adjusted
dense_t2 = np.linspace(0, 1, 5 * len(x_original_2))

#List of all points on smoothed curve (5 * original length)
x_smoothed_1 = spl_x1(dense_t1)
y_smoothed_1 = spl_y1(dense_t1)
z_smoothed_1 = spl_z1(dense_t1)
x_smoothed_2 = spl_x2(dense_t2)
y_smoothed_2 = spl_y2(dense_t2)
z_smoothed_2 = spl_z2(dense_t2)

'''Extracting 50 points from the smoothed curve, which contains five times the original length'''
x_fifty_smoothed_1 = []
y_fifty_smoothed_1 = []
z_fifty_smoothed_1 = []
x_fifty_smoothed_2 = []
y_fifty_smoothed_2 = []
z_fifty_smoothed_2 = []

#Get 50 points from the original smoothed curve data point list and add it to the above

#You can delete this part later but I'm just commenting this so the code makes more sense.
#Each list has a different number of data points so to get 50 data points from each list 
#that are about equidistant from each other, you divide the total length (on average, smoothed 
#curve has about 250) by 50, which gets you the distance between each data point,
#hence distance_between_pts variable. So you collect every nth data point which will get you about 50.
#The if statements are just to ensure that there are 50 data points.
dist_between_pts = len(x_smoothed_1) / 50
curr_index = 0.0
for index in range(len(x_smoothed_1)):
    if dist_between_pts <= 0:
        continue
    if (index != int(curr_index)) :
        continue
    if len(x_fifty_smoothed_1) == 50:
        continue
    x_fifty_smoothed_1.append(x_smoothed_1[index])
    y_fifty_smoothed_1.append(y_smoothed_1[index])
    z_fifty_smoothed_1.append(z_smoothed_1[index])
    curr_index += dist_between_pts

dist_between_pts = len(x_smoothed_2) / 50
curr_index = 0.0
for index in range(len(x_smoothed_2)):
    if dist_between_pts <= 0:
        continue
    if (index != int(curr_index)) :
        continue
    if len(x_fifty_smoothed_2) == 50:
        continue
    x_fifty_smoothed_2.append(x_smoothed_2[index])
    y_fifty_smoothed_2.append(y_smoothed_2[index])
    z_fifty_smoothed_2.append(z_smoothed_2[index])
    curr_index += dist_between_pts

'''Moving the strokes the origin based on bounding box'''
#Calculate bounding box for centering
center1 = ((np.max(x_fifty_smoothed_1) + np.min(x_fifty_smoothed_1))/2, 
           (np.max(y_fifty_smoothed_1) + np.min(y_fifty_smoothed_1))/2, 
           (np.max(z_fifty_smoothed_1) + np.min(z_fifty_smoothed_1))/2)
center2 = ((np.max(x_fifty_smoothed_2) + np.min(x_fifty_smoothed_2))/2, 
           (np.max(y_fifty_smoothed_2) + np.min(y_fifty_smoothed_2))/2, 
           (np.max(z_fifty_smoothed_2) + np.min(z_fifty_smoothed_2))/2)

#List of 50 points on smoothed curve centered at the origin
x_centered_smooth_1 = [val - center1[0] for val in x_fifty_smoothed_1]
y_centered_smooth_1 = [val - center1[1] for val in y_fifty_smoothed_1]
z_centered_smooth_1 = [val - center1[2] for val in z_fifty_smoothed_1]
x_centered_smooth_2 = [val - center2[0] for val in x_fifty_smoothed_2]
y_centered_smooth_2 = [val - center2[1] for val in y_fifty_smoothed_2]
z_centered_smooth_2 = [val - center2[2] for val in z_fifty_smoothed_2]

'''Plotting the strokes on the figures'''
# Draw traces
fig.add_trace(go.Scatter3d(
    x = x_centered_smooth_1,
    y = y_centered_smooth_1,
    z = z_centered_smooth_1,
    mode='markers',
    marker=dict(
        size=2,
        colorscale='sunset_r',  # colorscale
        opacity=0.8,
    ),
    name='Sub 16 Sess 2',
    showlegend=True)
)
fig.add_trace(go.Scatter3d(
    x = x_centered_smooth_2,
    y = y_centered_smooth_2,
    z = z_centered_smooth_2,
    mode='markers',
    marker=dict(
        size=2,
        colorscale='viridis',  # colorscale
        opacity=0.8,
    ),
    name='Sub 16 Sess 3',
    showlegend=True)
)
fig.update_layout(
    title_text='Pan Down (Right Unimanual)', 
    scene_aspectmode='data'
)
fig.show()


Overview: Takes in a data directory and stores it in a dictionary that is more efficient to iterate through for creating box plots. Rather than iterating by subject id number in the CleanedData directory, it will iterate by gesture.

Dictionary details:
    - Dictionary will have 13 keys to represent the 13 different gestures. 
    - Each key will have a value of a list that contains two lists. 
    - The two lists will represent the freeform and instruction folder respectively. 
    - Within each of those two lists, they will contain 37 data files paths. 
    - Ex: {'PanLeft': [../session_F_PanSelect_sub1ID.csv, ../session_F_PanSelect_sub2ID.csv, ..][../session_I_PanSelect_sub1ID.csv, ../session_I_PanSelect_sub2ID.csv, ..]}

Cleaned data directory must have the following subfolders/files: ...\CleanedData\Sub#\Session_Sub#_Sess#\\file.csv
The names that are filled in are:
    - '#' in Sub# will be the subject's number
    - 'Session' in Session_Sub#_Sess# is either Freeform or Instructional, '#' in Sub# is the subject's number, and '#' in Sess# is the session number
    - 'file' in file.csv is the name of the file

Requirements:
    - First code block must run successfully
        - Import libraries
    - Cleaned data directory must have the template path: ...\CleanedData\Sub#\Session_Sub#_Sess#\file.csv
        - CleanedData: fixed
        - Sub#: '#' is filled in with the subject's identifier number
        - Session_Sub#_Sess#: first '#' is filled in with the subject's identifier number and second '#' is filled in with the session number for that subject
        - file.csv: 'file' is filled in with the file's name
    - Edit cleaned_data_folder_path to your cleaned data directory path

In [3]:
''' Edit variable here'''
cleaned_data_folder_path = 'C:\\Users\\katie\\Desktop\\cpsc\\VRelax\\gestureInterface\\CleanedData'

gesture_dict = {}

# Iterates through CleanedData directory to make a dictionary of data where the 13 keys are gesture types and its values 
# are two lists representing freeform and instructional. Each list will contain the path to the 37 csv data files.
for root, sub_folders, files in os.walk(cleaned_data_folder_path):
    for file in files:
        # Splits file name for gesture and session type identification
        file_split = file.split("_")
        file_gesture = file_split[3]
        file_session_type = file_split[2]
         
        # Temporary: ignore box select and subject 1
        if file_gesture == 'BoxSelect':
            continue
        if file_split[5] == str(1):
            continue

        # Ignores the thank you files
        if (file_session_type != 'F') and (file_session_type != 'I'):
            continue

        # If a gesture is not in the dictionary, make the gesture a new key with list values freeform and instructional
        if file_gesture not in gesture_dict:
            freeform_folder = []
            instructional_folder = []
            gesture_dict[file_gesture] = [freeform_folder, instructional_folder]

        if file_session_type == 'F':
            gesture_dict[file_gesture][0].append(os.path.join(root, file))
        else:
            gesture_dict[file_gesture][1].append(os.path.join(root, file))

Overview: Plots all gesture strokes of the same template and the average of those.

Details:
    - Currently only works with pan down unimanual (right controller)

Requirements:
    - Must run the first code block successfully to import libraries.
    - Must run the third/above code block successfully to create gesture dictionary.

In [4]:
'''Creating a list of dataframes that will be plotted in the figure'''
#Adds dataframes that have the same template(pan down unimanual right controller) to a list.
all_dfs = []
pan_down_key = list(gesture_dict.keys())[0]
for file_path in gesture_dict[pan_down_key][0]:
    df = pd.read_csv(file_path)
    # Drops strokes that don't use right controller
    df.drop(df[(df["trigger_pull_amount_right"] == 0)].index, inplace=True)
    # Drops strokes where both controller are in use
    df.drop(df[(df["trigger_pull_amount_right"] == 1) & (df["trigger_pull_amount_left"] == 1)].index, inplace=True)
    # Drops strokes based on trial number
    df.drop(df[(df['gesture_counter_UI']) != 1].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    # To counter controllers that have a time difference between trigger pulls
    if (len(df) > 15):
        all_dfs.append(df)


#Plot all strokes (right controller data only) from all_dfs on single figure
fig = go.Figure()
stroke_num = 0
for df in all_dfs:
    stroke_num += 1
    fig.add_trace(go.Scatter3d(
    x=df['r_controller_translation_x'],
    y=df['r_controller_translation_y'],
    z=df['r_controller_translation_z'],
    mode='markers',
    marker=dict(
        size=2,
        color='blue',  # colorscale
        opacity=0.8,
    ),
    name='right'+str(stroke_num),
    showlegend=True))

    fig.add_trace(go.Scatter3d(
    x=df['l_controller_translation_x'],
    y=df['l_controller_translation_y'],
    z=df['l_controller_translation_z'],
    mode='markers',
    marker=dict(
        size=2,
        color='red',  # colorscale
        opacity=0.8,
    ),
    name='left'+str(stroke_num),
    showlegend=True))

'''Calculate average of all the strokes based on coordinates'''
#List will contain the different strokes represented by a tuple.
#Each tuple will contain three lists, each containing the X, Y, Z coordinates that make up that stroke.

fifty_pts = []  #Store 50 data points on the smoothed curve
for df in all_dfs:

    '''Smooth the strokes'''
    x=df['r_controller_translation_x']
    y=df['r_controller_translation_y']
    z=df['r_controller_translation_z']

    # The number of control points and knots
    k = 3  # degree of the B-spline
    t = np.linspace(0, 1, len(df))

    spl_x = make_interp_spline(t, x, k=k)
    spl_y = make_interp_spline(t, y, k=k)
    spl_z = make_interp_spline(t, z, k=k)

    # Evaluate the B-spline over a dense set of points for a smooth trajectory
    dense_t = np.linspace(0, 1, 5 * len(df))  # points in the smoothed curve; can be adjusted
    x_smooth = spl_x(dense_t)
    y_smooth = spl_y(dense_t)
    z_smooth = spl_z(dense_t)
    
    fifty_x_smooth = []
    fifty_y_smooth = []
    fifty_z_smooth = []
    dist_between_pts = len(x_smooth) / 50

    '''Extract fifty data points from the smoothed curve'''
    curr_index = 0.0
    for index in range(len(x_smooth)):
        if dist_between_pts <= 0:
            continue
        if (index != int(curr_index)) :
            continue
        if len(fifty_x_smooth) == 50:
            continue
        fifty_x_smooth.append(x_smooth[index])
        fifty_y_smooth.append(y_smooth[index])
        fifty_z_smooth.append(z_smooth[index])
        curr_index += dist_between_pts
    fifty_pts.append((fifty_x_smooth, fifty_y_smooth, fifty_z_smooth))

#Stores the x, y, z coordinates that make up the averaged stroke
averaged_x = []
averaged_y = []
averaged_z = []

#Accesses the x(0), y(1), z(2) coordinates
for i in range(3):
    #Accesses the index of the 50 data points
    for j in range(50):
        sum_coordinates = 0.0
        #Adds the value at j index of the 50 data point on the i coordinate to the sum
        for stroke in fifty_pts:
            sum_coordinates += stroke[i][j]    
        #Calculates the average value of the coordinate based on the sum and appends to the average stroke list
        if i == 0:
            averaged_x.append(sum_coordinates/len(fifty_pts))
        elif i == 1:
            averaged_y.append(sum_coordinates/len(fifty_pts))
        else:
            averaged_z.append(sum_coordinates/len(fifty_pts))

'''Plot the average stroke'''
fig.add_trace(go.Scatter3d(
x=averaged_x,
y=averaged_y,
z=averaged_z,
mode='markers',
marker=dict(
    size=2,
    color='yellow',  # colorscale
    opacity=0.8,
),
name='average right controller',
showlegend=True))

fig.update_layout(
    title_text='Pan Down (Right Controller Used)', 
    scene_aspectmode='data'
)
fig.show()
